In [670]:
import urllib
import shutil
import pandas as pd
import sys
from pathlib import Path
import zipfile
import requests
import numpy as np

In [671]:
date='120118'
fy='2016-17'

In [672]:
file_zip='EQ'+date+'_CSV.ZIP'
file='EQ'+date+'.CSV'
base_url='http://www.bseindia.com/download/BhavCopy/Equity/'+file_zip
purchase_data='purchase_data_'+fy

In [673]:
purchase_data

'purchase_data_2016-17'

In [674]:
def del_tmp_files():
    for p in Path(".").glob("EQ*"):
        p.unlink()


In [675]:
def is_downloadable(url):
    """
    Does the url contain a downloadable resource
    """
    h = requests.head(url, allow_redirects=True)
    header = h.headers
    content_type = header.get('content-type')
    if 'text' in content_type.lower():
        return False
    if 'html' in content_type.lower():
        return False
    return True

In [676]:
return_value = is_downloadable(base_url)

In [677]:
return_value

True

In [678]:
if return_value:
    request = urllib.request.Request(base_url, headers={'User-Agent': 'Mozilla/5.0'})
    with urllib.request.urlopen(base_url) as response, open(file_zip, 'wb') as out_file:
        shutil.copyfileobj(response, out_file)

In [679]:
#unzip file
with zipfile.ZipFile(file_zip,"r") as zip_ref:
    zip_ref.extractall()

In [680]:
#read csv file
bhav_df = pd.read_csv(file,header=0)
bhav_filt_df = bhav_df.loc[:, ['SC_CODE', 'SC_NAME', 'CLOSE']]

In [681]:
purchase_df = pd.read_csv('/home/oracle/StockAnalysis/'+purchase_data+'.csv', header=0)
purchase_df_buy = purchase_df[purchase_df['Ind'] == 'B']
purchase_df_sell = purchase_df[purchase_df['Ind'] == 'S']

#join both pandas df
merged_df_buy = pd.merge(bhav_filt_df, purchase_df_buy, on='SC_CODE', how='inner')
merged_df_sell = pd.merge(bhav_filt_df, purchase_df_sell, on='SC_CODE', how='inner')


In [682]:
frames = [ merged_df_buy, merged_df_sell]
result = pd.concat(frames).reset_index()
result

,index,SC_CODE,SC_NAME,CLOSE,TransactionDate,CompanyName,SharesUnits,UnitPrice,net_brokerage,Ind
0,0,500380,JK LAKSHMI,443.00,15-11-2016,JK Lakshmi Cement,5,372.250000,6.36,B
1,1,500495,ESCORTS LTD.,811.00,2/6/2016,Escorts,51,173.000000,47.16,B
2,2,505710,GRAUER& WEIL,79.95,15-11-2016,Grauer & Weil,50,35.000000,38.61,B
3,3,505710,GRAUER& WEIL,79.95,22-6-2016,Grauer & Weil,146,27.050000,22.03,B
4,4,506590,PHIL.CARBON,1453.40,15-11-2016,Phillips Carbon,3,208.250000,12.72,B
5,5,506943,J.B.CHEMICAL,338.15,10/8/2016,JB Chem. & Pharma,57,283.200000,37.38,B
6,6,506943,J.B.CHEMICAL,338.15,26-7-2016,JB Chem. & Pharma,57,270.500000,60.75,B
7,7,508814,COSMO FILMS,377.15,26-7-2016,Cosmo Films,7,361.000000,35.15,B
8,8,508814,COSMO FILMS,377.15,29-4-2016,Cosmo Films,17,382.140000,46.52,B
9,9,511072,DEWAN H.FIN.,613.45,15-11-2016,DHFL,10,234.960000,18.59,B


In [683]:
#filter specific columns
filt_df = result.loc[:, ['TransactionDate' , 'SC_CODE' , 'SC_NAME' , 
                            'SharesUnits' , 'UnitPrice' , 'net_brokerage' , 'CLOSE', 'Ind' ]]
#rename df columns
df = filt_df.rename(columns={'SC_CODE': 'CompanyCode', 
                             'SC_NAME': 'ScriptName', 'CLOSE': 'CurrentPrice'})
    
    
df_bought = df.loc[df['Ind'] == 'B']
df_sold = df.loc[df['Ind'] == 'S']

In [684]:
#define CAGR
def CAGR(MarketValue, TotalCost, periods):
    try:
        cagr = (((MarketValue/TotalCost)**(1/periods)-1)*100)
    except Exception as err:
        #print ("=" * 80 + "\nSomething went wrong while calculating CAGR: {}\n".format(err) + "=" * 80)
        return 0
    else:
        return cagr

#define gain_loss_percent
def gain_loss_per(Gain_Loss, TotalCost):
    try:
        gain_loss_per = Gain_Loss / TotalCost
    except Exception as err:
        return 0
    else:
        return gain_loss_per

""" 
https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.select.html
numpy.select(condlist, choicelist, default=0)[source]
"""
def totalCost(SharesUnits, UnitPrice, net_brokerage):
    return (SharesUnits * UnitPrice) + net_brokerage

def marketPrice(Ind, CurrentPrice, UnitPrice, SharesUnits):
    #m1 = Ind=='B'
    #m2 = Ind=='S'
    #a = SharesUnits * CurrentPrice
    #b = SharesUnits * UnitPrice
    #return np.select([m1, m2], [a,b], default=0)
    if Ind=='B':
        price = SharesUnits * CurrentPrice
    else:
        price = SharesUnits * UnitPrice
    return price

def durationMonths(Sell_Ind, CurrentDate, PurchaseDate, SellDate):
    m1 = Sell_Ind=='N'
    m2 = Sell_Ind=='Y'
    a = CurrentDate.to_period(freq='M') - PurchaseDate.to_period(freq='M')
    b = SellDate.to_period(freq='M') - PurchaseDate.to_period(freq='M')
    return np.select([m1, m2], [a,b], default=0)


In [685]:
pd.options.mode.chained_assignment = None  # default='warn'
    
#metrics bought stocks
df_bought['TotalCost'] = df_bought.apply(lambda row: totalCost(row['SharesUnits'],row['UnitPrice'], row['net_brokerage']), axis=1).astype(float).round(2)
df_bought['MarketValue'] =  (df_bought.apply(lambda row: marketPrice(row['Ind'], row['CurrentPrice'], row['UnitPrice'], row['SharesUnits']), axis=1)).astype(float).round(2)
df_bought['Gain_Loss'] = df_bought.apply(lambda row: row.MarketValue - row.TotalCost, axis=1).round(2)
df_bought['Gain_Loss(%)'] = df_bought.apply(lambda row: gain_loss_per(row['Gain_Loss'], row['TotalCost']), axis=1).round(2)
df_bought['CurrentDate'] = pd.to_datetime('today')
df_bought['PurchaseDate'] =  pd.to_datetime(df_bought['TransactionDate'])

df_bought_fil = df_bought.rename(columns={'UnitPrice': 'Inv.Price', 'net_brokerage': 'Brkg.', 'CLOSE': 'CurrentPrice'})
df_B = df_bought_fil.loc[:, ['CompanyCode', 'ScriptName', 'SharesUnits', 'Inv.Price', 'Brkg.', 'CurrentPrice', 'TotalCost', 
                      'MarketValue']]


#metrics sold stocks
df_sold['TotalCost'] = df_sold.apply(lambda row: totalCost(row['SharesUnits'],row['UnitPrice'], row['net_brokerage']), axis=1).astype(float).round(2)
df_sold['MarketValue'] =  (df_sold.apply(lambda row: marketPrice(row['Ind'], row['CurrentPrice'], row['UnitPrice'], row['SharesUnits']), axis=1)).astype(float).round(2)
df_sold['Gain_Loss'] = df_sold.apply(lambda row: row.MarketValue - row.TotalCost, axis=1).round(2)
df_sold['Gain_Loss(%)'] = df_sold.apply(lambda row: gain_loss_per(row['Gain_Loss'], row['TotalCost']), axis=1).round(2)
df_sold['CurrentDate'] = pd.to_datetime('today')
df_sold['SoldDate'] =  pd.to_datetime(df_sold['TransactionDate'])

df_sold_fil = df_sold.rename(columns={'UnitPrice': 'SellPrice', 'net_brokerage': 'Brkg.', 'CLOSE': 'CurrentPrice'})  
df_S = df_sold_fil.loc[:, ['CompanyCode', 'SharesUnits', 'SellPrice', 'Brkg.', 'SoldDate']]

In [686]:
df_S

,CompanyCode,SharesUnits,SellPrice,Brkg.,SoldDate
29,500380,5,475.100000,17.73,2017-05-23
30,500495,51,259.116667,58.98,2016-09-08
31,517224,54,1.420000,10.85,2017-10-03
32,520043,29,200.000000,31.35,2017-06-21
33,521200,55,148.500000,36.98,2016-05-30
34,532155,20,370.000000,35.50,2016-03-06
35,532637,9,172.000000,13.15,2017-07-14
36,532673,155,12.250000,24.51,2016-10-08
37,532673,600,11.500000,35.51,2016-11-08
38,532760,50,172.000000,53.78,2016-04-29


In [687]:
def buy_agg(x):
    names = {
        'Purchased_Units': x['SharesUnits'].sum(),
        'Inv.Price':  x['Inv.Price'].mean(),
        'TotalCost': x['SharesUnits'].sum() * x['Inv.Price'].mean() + x['Brkg.'].sum(),
        'CurrentPrice': x['CurrentPrice'].mean(),
        'MarketValue': x['CurrentPrice'].mean() * x['SharesUnits'].sum()}
    return pd.Series(names, index=['Purchased_Units', 'Inv.Price', 'TotalCost','CurrentPrice', 'MarketValue'])

aggregated_buy_df = df_B.groupby(["CompanyCode", "ScriptName"]).apply(buy_agg).reset_index()

def sold_agg(x):
    names = {
        'Sold_Units': x['SharesUnits'].sum(),
        'SellPrice':  x['SellPrice'].mean(),
        'TotalSellingPrice': x['SharesUnits'].sum() * x['SellPrice'].mean() + x['Brkg.'].sum()}
    return pd.Series(names, index=['Sold_Units', 'SellPrice', 'TotalSellingPrice'])

aggregated_sold_df = df_S.groupby(["CompanyCode"]).apply(sold_agg).reset_index()


In [688]:
aggregated_sold_df


,CompanyCode,Sold_Units,SellPrice,TotalSellingPrice
0,500380,5.0,475.100000,2393.230000
1,500495,51.0,259.116667,13273.930002
2,517224,54.0,1.420000,87.530000
3,520043,29.0,200.000000,5831.350000
4,521200,55.0,148.500000,8204.480000
5,532155,20.0,370.000000,7435.500000
6,532637,9.0,172.000000,1561.150000
7,532673,755.0,11.875000,9025.645000
8,532760,50.0,172.000000,8653.780000
9,533192,122.0,36.800000,4513.470000


In [689]:
merged_df = pd.merge(aggregated_buy_df, aggregated_sold_df, on='CompanyCode', how='left')
merged_df['Sold_Units'] = merged_df['Sold_Units'].fillna(0)
merged_df['SellPrice'] = merged_df['SellPrice'].astype(float).round(2).fillna(0)
merged_df['TotalSellingPrice'] = merged_df['TotalSellingPrice'].astype(float).round(2).fillna(0)
merged_df['Inv.Price'] = merged_df['Inv.Price'].astype(float).round(2).fillna(0)
merged_df['TotalCost'] = merged_df['TotalCost'].astype(float).round(2).fillna(0)


In [690]:
merged_df

,CompanyCode,ScriptName,Purchased_Units,Inv.Price,TotalCost,CurrentPrice,MarketValue,Sold_Units,SellPrice,TotalSellingPrice
0,500380,JK LAKSHMI,5.0,372.25,1867.61,443.00,2215.00,5.0,475.10,2393.23
1,500495,ESCORTS LTD.,51.0,173.00,8870.16,811.00,41361.00,51.0,259.12,13273.93
2,505710,GRAUER& WEIL,196.0,31.02,6141.54,79.95,15670.20,0.0,0.00,0.00
3,506590,PHIL.CARBON,3.0,208.25,637.47,1453.40,4360.20,0.0,0.00,0.00
4,506943,J.B.CHEMICAL,114.0,276.85,31659.03,338.15,38549.10,0.0,0.00,0.00
5,508814,COSMO FILMS,24.0,371.57,8999.35,377.15,9051.60,0.0,0.00,0.00
6,511072,DEWAN H.FIN.,10.0,234.96,2368.19,613.45,6134.50,0.0,0.00,0.00
7,517224,SUJANA INDUS,54.0,1.97,145.73,1.16,62.64,54.0,1.42,87.53
8,520043,MUNJAL SHOWA,29.0,182.19,5317.01,298.25,8649.25,29.0,200.00,5831.35
9,521064,TRIDENT,59.0,78.00,4628.34,89.20,5262.80,0.0,0.00,0.00


In [691]:
def gain_loss(TotalSellingPrice, TotalCost, MarketValue):
    #m1 = Ind=='B'
    #m2 = Ind=='S'
    #a = SharesUnits * CurrentPrice
    #b = SharesUnits * UnitPrice
    #return np.select([m1, m2], [a,b], default=0)
    if TotalSellingPrice == 0.0:
        gain_loss = MarketValue -  TotalCost
    else:
        gain_loss = TotalSellingPrice - TotalCost
    return gain_loss

#define gain_loss_percent
def gain_loss_per(OverallGain, TotalCost):
    try:
        gain_loss_per = OverallGain / TotalCost
    except Exception as err:
        return 0
    else:
        return gain_loss_per

In [692]:
merged_df['OverallGain'] = merged_df.apply(lambda row: gain_loss(row['TotalSellingPrice'],row['TotalCost'], row['MarketValue']), axis=1).astype(float).round(2)
merged_df['OverallGain(%)'] = merged_df.apply(lambda row: gain_loss_per(row['OverallGain'], row['TotalCost']), axis=1).round(2)


def actual_value(TotalSellingPrice, MarketValue):
    if TotalSellingPrice == 0.00:
        actual_value = MarketValue
    else:
        actual_value = TotalSellingPrice
    return actual_value
merged_df['actual_value'] = merged_df.apply(lambda row: actual_value(row['TotalSellingPrice'], row['MarketValue']), axis=1).astype(float).round(2)


In [693]:
import decimal


#Total_TotalCost = merged_df['TotalCost'].sum()
#a = decimal.Decimal(Total_TotalCost)
Total_TotalCost = (round(decimal.Decimal(merged_df['TotalCost'].sum()),2))
Total_MarketValue = (round(decimal.Decimal(merged_df['actual_value'].sum()),2))
Total_OverallGain = Total_MarketValue - Total_TotalCost
Total_OverallGain_per = Total_OverallGain / Total_TotalCost


In [694]:
final_df = merged_df.drop(['actual_value'], axis=1)

In [695]:
# Formatting the data
from xlsxwriter.utility import xl_rowcol_to_cell

writer = pd.ExcelWriter("PortfolioAnalysis_"+fy+".xlsx", engine='xlsxwriter')
final_df.to_excel(writer, sheet_name='report', index=False)
#aggregated_df.to_excel(writer, sheet_name='aggregated_report', index=False)
workbook = writer.book

# Add a number format for cells with money.
money_fmt = workbook.add_format({'num_format': '₹#,##0.00', 'bold': True})

# Add a percent format with 1 decimal point
percent_fmt = workbook.add_format({'num_format': '0.00%', 'bold': True})

#Date Format
date_fmt = workbook.add_format({'num_format': 'yyyy-mm-dd'})


# Add a format. Light red fill with dark red text for -ve values
format1 = workbook.add_format({'bg_color': '#FFC7CE',
                               'font_color': '#9C0006'})

# Add a format. Green fill with dark green text for +ve values
format2 = workbook.add_format({'bg_color': '#C6EFCE',
                               'font_color': '#006100'})

# Create a format to use in the merged range.
merge_format1 = workbook.add_format({
    'bold': 0.5,
    'border': 1,
    'align': 'center',
    'valign': 'vcenter',
    'fg_color': 'yellow',
    'num_format': '₹#,##0.00'})

merge_format2 = workbook.add_format({
    'bold': 0.5,
    'border': 1,
    'align': 'center',
    'valign': 'vcenter',
    'fg_color': 'yellow',
    'num_format': '0.00%'})

#worksheet1
worksheet1 = writer.sheets['report']
worksheet1.set_zoom(90)

# set width of columns
worksheet1.set_column('A:H', 15)
worksheet1.set_column('H:L', 25)
#worksheet1.set_column('B:E', 20)
#worksheet1.set_column('N:O', 20)

# money_fmt columns
worksheet1.set_column('D:G', 12, money_fmt)
worksheet1.set_column('I:K', 20, money_fmt)
# percent_fmt columns
worksheet1.set_column('L:L', 12, percent_fmt)
#Date Columns
#worksheet1.set_column('D:E', 20, date_fmt)
#worksheet1.set_column('N:N', 20, date_fmt)

# Define our range for the color formatting
number_rows = len(final_df.index)
color_range = "K2:L{}".format(number_rows1+1)
#active_stocks = "A2:B{}".format(number_rows1+1)

# Highlight the -ve  values in Red
worksheet1.conditional_format(color_range, {'type': 'cell',
                                           'criteria': '<',
                                           'value': '0',
                                           'format': format1})

# Highlight the +ve values in Green
worksheet1.conditional_format(color_range, {'type': 'cell',
                                           'criteria': '>=',
                                           'value': '0',
                                           'format': format2})

'''worksheet1.conditional_format(active_stocks, {'type': 'cell',
                                           'criteria': 'H2:H{}.format(number_rows1+1) > 0',
                                           'value': '0',
                                           'format': format2})
'''
end_row=number_rows+1
# Merge 3 cells.
worksheet1.merge_range('A'+str(end_row+1)+':E'+str(end_row+1), 
                       '2016-2017 Investment: '+ str(Total_TotalCost), merge_format1)

worksheet1.merge_range('F'+str(end_row+1)+':G'+str(end_row+1), 
                       'LatestValue: '+ str(Total_MarketValue), merge_format1)

worksheet1.merge_range('I'+str(end_row+1)+':J'+str(end_row+1), 
                       'OverallGain: '+ str(Total_OverallGain), merge_format1)

worksheet1.merge_range('K'+str(end_row+1)+':L'+str(end_row+1),Total_OverallGain_per, merge_format2)



writer.save() 